In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import re 
import json

In [2]:
game_dataset = pd.read_csv('dataset/games_dropped_column.csv')

In [3]:
game_dataset["release_date"] = game_dataset["release_date"].apply(lambda x: pd.to_datetime(x).strftime('%Y'))

In [4]:
def process_genres(value):
    if isinstance(value, float):
        return []
    genres = value.split(",")
    genres = [genre.strip() for genre in genres]
    return genres

game_dataset["genres"] = game_dataset["genres"].apply(process_genres)

In [5]:
all_genres = []
for genres in game_dataset["genres"]:
    for elem in genres:
        all_genres.append(elem)

all_genres_distributions = pd.Series(all_genres).value_counts()
all_genres_distributions_20 = all_genres_distributions.head(20)

genres_20 = all_genres_distributions_20.keys().to_list()

print(genres_20)

['Indie', 'Casual', 'Action', 'Adventure', 'Simulation', 'Strategy', 'RPG', 'Early Access', 'Free to Play', 'Sports', 'Racing', 'Massively Multiplayer', 'Utilities', 'Design & Illustration', 'Violent', 'Animation & Modeling', 'Education', 'Free To Play', 'Video Production', 'Gore']


In [6]:
bar_dict = []
for genre in genres_20:
    bar_dict.append({"genre": genre, "nb_game": np.zeros(2025-1997+1), "nb_hour_played": np.zeros(2025-1997+1) })

bar_dict = {elem["genre"]: elem for elem in bar_dict}

In [12]:
game_dataset_ordered

,id,name,release_date,Estimated owners,user_score,average_playtime_forever,median_playtime_forever,Publishers,genres
22217,232770,POSTAL,1997,5000000 - 10000000,0,215,223,Running With Scissors,"[Action, Indie]"
36165,282010,Carmageddon Max Pack,1997,100000 - 200000,0,39,63,THQ Nordic,"[Action, Indie, Racing]"
49557,70,Half-Life,1998,5000000 - 10000000,0,459,127,Valve,[Action]
17218,224920,Legacy of Kain: Soul Reaver,1999,500000 - 1000000,0,267,375,Square Enix,"[Action, Adventure]"
19555,20,Team Fortress Classic,1999,5000000 - 10000000,0,143,23,Valve,[Action]
...,...,...,...,...,...,...,...,...,...
89023,2879000,Atman:Rebel Flame,2024,0 - 0,0,0,0,JUP GAME,"[Casual, Indie, Strategy, Free to Play]"
89015,2878360,Save and Survive,2024,0 - 20000,0,0,0,Hede,"[Action, Adventure, Casual, Indie, Racing, RPG..."
97409,3054200,Lober Lobe,2024,0 - 0,0,0,0,Trash Vomit Studios,"[Action, Casual, Indie]"
95850,3109040,Refugees: Shelter,2025,0 - 0,0,0,0,Roche Limit Studios,"[Adventure, Indie, RPG, Simulation]"


In [7]:
game_dataset_ordered = game_dataset.sort_values(by="release_date", ascending=True)
game_dataset_ordered

def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

for i in range(len(game_dataset_ordered)):
    game = game_dataset_ordered.iloc[i]
    if len(intersection(game["genres"], genres_20)) > 0:
        for genre in intersection(game["genres"], genres_20):
            bar_dict[genre]["nb_game"][int(game["release_date"])-1997] += 1
            bar_dict[genre]["nb_hour_played"][int(game["release_date"])-1997] += game["average_playtime_forever"] 

bar_dict
        

{'Indie': {'genre': 'Indie',
  'nb_game': array([2.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00,
         2.000e+00, 0.000e+00, 3.000e+00, 1.100e+01, 8.000e+00, 2.600e+01,
         6.700e+01, 7.000e+01, 1.080e+02, 1.740e+02, 2.760e+02, 9.180e+02,
         1.934e+03, 3.095e+03, 4.480e+03, 6.220e+03, 5.922e+03, 7.144e+03,
         8.186e+03, 8.729e+03, 9.380e+03, 7.744e+03, 1.000e+00]),
  'nb_hour_played': array([2.54000e+02, 0.00000e+00, 0.00000e+00, 0.00000e+00, 4.86000e+02,
         0.00000e+00, 3.04000e+02, 0.00000e+00, 8.00000e+01, 1.19070e+04,
         3.09000e+02, 4.06400e+03, 7.98800e+03, 1.35470e+04, 2.75990e+04,
         4.39300e+04, 8.24440e+04, 2.30456e+05, 5.11798e+05, 5.57225e+05,
         4.26744e+05, 5.02451e+05, 4.21545e+05, 3.56680e+05, 3.87124e+05,
         1.57896e+05, 6.86210e+04, 7.60800e+03, 0.00000e+00])},
 'Casual': {'genre': 'Casual',
  'nb_game': array([0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
         1.000e+00, 0.000

In [8]:
def applyCumulativeSum(arr):
    return np.cumsum(arr, dtype=int)

bar_dict_cumu = bar_dict.copy()
"""
for genre in bar_dict:
    bar_dict_cumu[genre]["nb_game"] = applyCumulativeSum(bar_dict[genre]["nb_game"])
    bar_dict_cumu[genre]["nb_hour_played"] = applyCumulativeSum(bar_dict[genre]["nb_hour_played"])
"""

'\nfor genre in bar_dict:\n    bar_dict_cumu[genre]["nb_game"] = applyCumulativeSum(bar_dict[genre]["nb_game"])\n    bar_dict_cumu[genre]["nb_hour_played"] = applyCumulativeSum(bar_dict[genre]["nb_hour_played"])\n'

In [9]:
# convert np array to list
for genre in bar_dict:
    bar_dict_cumu[genre]["nb_game"] = bar_dict_cumu[genre]["nb_game"].tolist()
    bar_dict_cumu[genre]["nb_hour_played"] = bar_dict_cumu[genre]["nb_hour_played"].tolist()

In [10]:
final = []
for genre in bar_dict_cumu:
    final.append({"genre": genre, "nb_game": bar_dict_cumu[genre]["nb_game"], "nb_hour_played": bar_dict_cumu[genre]["nb_hour_played"]})

final

[{'genre': 'Indie',
  'nb_game': [2.0,
   0.0,
   0.0,
   0.0,
   1.0,
   0.0,
   2.0,
   0.0,
   3.0,
   11.0,
   8.0,
   26.0,
   67.0,
   70.0,
   108.0,
   174.0,
   276.0,
   918.0,
   1934.0,
   3095.0,
   4480.0,
   6220.0,
   5922.0,
   7144.0,
   8186.0,
   8729.0,
   9380.0,
   7744.0,
   1.0],
  'nb_hour_played': [254.0,
   0.0,
   0.0,
   0.0,
   486.0,
   0.0,
   304.0,
   0.0,
   80.0,
   11907.0,
   309.0,
   4064.0,
   7988.0,
   13547.0,
   27599.0,
   43930.0,
   82444.0,
   230456.0,
   511798.0,
   557225.0,
   426744.0,
   502451.0,
   421545.0,
   356680.0,
   387124.0,
   157896.0,
   68621.0,
   7608.0,
   0.0]},
 {'genre': 'Casual',
  'nb_game': [0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   1.0,
   0.0,
   0.0,
   19.0,
   12.0,
   42.0,
   69.0,
   78.0,
   48.0,
   56.0,
   89.0,
   428.0,
   786.0,
   1633.0,
   2507.0,
   3619.0,
   3269.0,
   4201.0,
   5173.0,
   5691.0,
   6371.0,
   5216.0,
   0.0],
  'nb_hour_played': [0.0,
   0.0,
   0.0,
   0.0,
 

In [11]:
with open("my-app/src/assets/bar2.json", "w", encoding="utf-8") as f:
    json.dump(final, f, indent=4, ensure_ascii=False)